In [ ]:
pip install patchify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from torchvision import models, transforms, datasets
import torch
import os
import torch.nn.functional as F
import cv2
from patchify import patchify
import numpy as np
import shutil
from PIL import Image
import torch.optim as optim
import torch.nn as nn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Para saber la etiqueta de las imágenes que se están probando es necesario ejecutar este código y tener acceso al conjunto de datos de BreakHis
# (tarda unos 15 min)

# Variables globales
magnification = "40X"
base_dataset = "drive/MyDrive/BreaKHis_v1/breast"
class_dir = ['benign', 'malignant']
sub_class_benign = ['adenosis', 'fibroadenoma', 'phyllodes_tumor', 'tubular_adenoma']
sub_class_malignant = ['ductal_carcinoma', 'lobular_carcinoma', 'mucinous_carcinoma', 'papillary_carcinoma']

# Recorremos directorios para obtener las rutas de las imágenes
image_paths = []
for class_type in class_dir:
    if class_type == 'benign':
        for subclass in sub_class_benign:
            cur_dir = os.path.join(base_dataset, class_type, 'SOB', subclass)
            for folder in os.listdir(cur_dir):
                cur_dir2 = os.path.join(base_dataset, class_type, 'SOB', subclass, folder, magnification)
                for file in os.listdir(cur_dir2):
                    image_paths.append(os.path.join(cur_dir2, file))
    else:
        for subclass in sub_class_malignant:
            cur_dir = os.path.join(base_dataset, class_type, 'SOB', subclass)
            for folder in os.listdir(cur_dir):
                cur_dir2 = os.path.join(base_dataset, class_type, 'SOB', subclass, folder, magnification)
                for file in os.listdir(cur_dir2):
                    image_paths.append(os.path.join(cur_dir2, file))

# Definimos array de imágenes y etiquetas
images = []
labels = []
for path in image_paths:
    if ".png" in path:
        image = cv2.imread(path, 1)
        images.append(image)
        labels.append(0) if "benign" in path else labels.append(1)


In [ ]:
# Función test
def test(model, device, optimizer, folder, num_imagen):

  data_transforms = {
      'test': transforms.Compose([
          transforms.ToTensor()
      ])}
    
  image_dataset = datasets.ImageFolder(folder, data_transforms['test'])
  dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=4, shuffle=True, num_workers=4)
  dataset_size = len(image_dataset)

  model.eval()
  corrects = 0
  running_loss = 0
  cnt = 0
  patches_preds = []
  
  with torch.no_grad():
    for inputs, classes in dataloader:
      cnt += 1
      inputs = inputs.to(device)

      outputs = model(inputs)
      _, predicted = torch.max(outputs.data, 1)
      predicted = predicted.cpu().numpy()

      for i in predicted:
        patches_preds.append(i)
   
    return labels[int(num_imagen)], patches_preds


      


In [ ]:
# test para una imagen en concreto
num_imagen = 390
image_path = '/content/drive/MyDrive/data/imagenes-test/im-' + str(num_imagen) + '.png'
image = cv2.imread(image_path, 1)

patches_data = patchify(image, (224,224,3), step=112)

patches = []
cnt = 0

for x in patches_data:
  cnt += 1
  for y in patches_data[cnt-1]:

    patches.append(y[0])

n = 0
for im in patches:
  n += 1
  data = Image.fromarray(im)
  imagepath = str('/content/drive/MyDrive/data/patches-test/test/pat-' + str(n) + '.png')
  data.save(imagepath)

model_path = "/content/drive/MyDrive/data/resnet20epoch0.001LR-state_dict-new.pth"
patches_path = '/content/drive/MyDrive/data/patches-test'

model = models.resnet18()
model.load_state_dict(torch.load(model_path))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

test(model, device, optimizer, patches_path, num_imagen)


In [ ]:
# test para todas las imágenes de prueba
images_test = "/content/drive/MyDrive/data/imagenes-test"
files = os.listdir(images_test)

model_path = "/content/drive/MyDrive/data/resnet20epoch0.001LR-state_dict-new.pth"
patches_path = '/content/drive/MyDrive/data/patches-test'

model = models.resnet18()
model.load_state_dict(torch.load(model_path))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

vp = 0
vn = 0
fp = 0
fn = 0

for image_path in files:
  path = image_path.split(sep='-')
  path = path[1].split('.')
  num = path[0]
  image_path = images_test + "/" + image_path
  image = cv2.imread(image_path, 1)

  patches_data = patchify(image, (224,224,3), step=112)

  patches = []
  cnt = 0

  for x in patches_data:
    cnt += 1
    for y in patches_data[cnt-1]:

      patches.append(y[0])

  
  n = 0
  for im in patches:
    n += 1
    data = Image.fromarray(im)
    imagepath = str('/content/drive/MyDrive/data/patches-test/test/pat-' + str(n) + '.png')
    data.save(imagepath)
    
  

  label , preds = test(model, device, optimizer, patches_path, num)


  numPatchesMalignos = sum(preds)
  numPatches = len(preds)
  umbral = 0.90

  if numPatchesMalignos/numPatches > umbral:
    print("predicción: 1, real: " + str(label))
    if label == 1:
      vp += 1
    else:
      fp += 1
  else:
    print("predicción: 0, real: " + str(label))
    if label == 1:
      fn += 1
    else:
      vn += 1

print("VP: " + str(vp))
print("VN: " + str(vn))
print("FP: " + str(fp))
print("FN: " + str(fn))


predicción: 0, real: 0
predicción: 0, real: 0
predicción: 0, real: 0
predicción: 1, real: 0
predicción: 0, real: 0
predicción: 0, real: 0
predicción: 0, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 0, real: 0
predicción: 0, real: 0
predicción: 0, real: 0
predicción: 1, real: 0
predicción: 0, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 0, real: 0
predicción: 0, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 0, real: 0
predicción: 0, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 0, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 1, real: 0
predicción: 0, real: 0
predicción: